In [4]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA,TruncatedSVD,NMF
from sklearn.preprocessing import Normalizer
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.options.mode.chained_assignment = None
import time
import numba

In [5]:
@numba.jit(nopython=True)
def year_binner(year,val=10):
    return year - year%val


In [71]:
heads=pd.read_pickle("/data/dharp/compounding/datasets/heads.pkl")
heads=heads.head(1_000_000)
heads.reset_index(inplace=True)
heads.year=heads.year.astype("int32")
heads=heads.query('1800 <= year <= 2010').copy()
heads.columns=['common','context','year','count']
heads['common']=heads['common'].str.replace(r'_noun$', r'_h', regex=True)
heads['time']=year_binner(heads['year'].values,10)
heads=heads.groupby(['common','context','time'])['count'].sum()
heads.head()

common  context  time
a_h     '_noun   1860    2.000
                 1870    4.000
                 1880    3.000
                 1890   12.000
                 1900   23.000
Name: count, dtype: float64

In [72]:
modifiers=pd.read_pickle("/data/dharp/compounding/datasets/modifiers.pkl")
modifiers=modifiers.head(1_000_000)
modifiers.reset_index(inplace=True)
modifiers.year=modifiers.year.astype("int32")
modifiers=modifiers.query('1800 <= year <= 2010').copy()
modifiers.columns=['common','context','year','count']
modifiers['common']=modifiers['common'].str.replace(r'_noun$', r'_m', regex=True)
modifiers['time']=year_binner(modifiers['year'].values,10)
modifiers=modifiers.groupby(['common','context','time'])['count'].sum()
modifiers.head()

common  context  time
a_m     '_noun   1820    1.000
                 1830    5.000
                 1840    7.000
                 1850    3.000
                 1870   12.000
Name: count, dtype: float64

In [73]:
compounds=pd.read_pickle("/data/dharp/compounding/datasets/compounds.pkl")
compounds=compounds.head(1_000_000)
compounds.reset_index(inplace=True)
compounds.year=compounds.year.astype("int32")
compounds=compounds.query('1800 <= year <= 2010').copy()
compounds['common']=compounds['modifier']+" "+compounds['head']
compounds['time']=year_binner(compounds['year'].values,10)
compounds=compounds.groupby(['common','context','time'])['count'].sum()
compounds.head()

common             context  time
a_noun aaaaa_noun  a_noun   1870   1.000
                            1910   1.000
                            1920   3.000
                            1930   1.000
                            1940   1.000
Name: count, dtype: float64

In [75]:
df=pd.concat([heads,modifiers,compounds], sort=True)
df.head()

common  context  time
a_h     '_noun   1860    2.000
                 1870    4.000
                 1880    3.000
                 1890   12.000
                 1900   23.000
Name: count, dtype: float64

In [76]:
df=df.to_sparse()

In [77]:
df, rows, _ = df.to_coo(row_levels=['common','time'],column_levels=['context'],sort_labels=False)

In [80]:
df_svd = NMF(n_components=300, random_state=1991)
display(df_svd.fit(df).explained_variance_ratio_.sum())
df_list=df_svd.fit(df).explained_variance_ratio_
df_reduced = df_svd.fit_transform(df)
df_reduced = Normalizer(copy=False).fit_transform(df_reduced)
df_reduced=pd.DataFrame(df_reduced,index=rows)
#df_reduced.reset_index(inplace=True)
df_reduced.head()

AttributeError: 'NMF' object has no attribute 'explained_variance_ratio_'

In [81]:
df_svd

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
  n_components=300, random_state=1991, shuffle=False, solver='cd',
  tol=0.0001, verbose=0)

In [66]:
df_reduced.index = pd.MultiIndex.from_tuples(df_reduced.index, names=['common', 'decade'])

In [67]:
df_reduced.shape

(19341, 300)

In [32]:
pd.Series(constituents_list).cumsum().plot(figsize=[15,15])

NameError: name 'constituents_list' is not defined

In [26]:
df_reduced.reset_index(inplace=True)

In [42]:
1745+348

2093

In [71]:
df_reduced.loc[df_reduced.index.get_level_values(0).str.endswith(r'_m')]

0     1      2      3      4      5      6      7    \
common         decade                                                         
a_m            1800   0.000 0.243 -0.141  0.050  0.658  0.073  0.008 -0.042   
               1810   0.001 0.165 -0.095  0.037  0.409  0.087  0.010 -0.035   
               1820   0.000 0.065 -0.038  0.013  0.175  0.021  0.003 -0.011   
               1830   0.000 0.033 -0.020  0.007  0.124 -0.015 -0.000 -0.002   
               1840   0.001 0.026 -0.011  0.006  0.093 -0.013 -0.002 -0.001   
               1850   0.000 0.045 -0.021  0.010  0.131  0.002  0.007 -0.006   
               1860   0.000 0.045 -0.024  0.010  0.128  0.008  0.005 -0.007   
               1870   0.000 0.061 -0.035  0.014  0.128  0.050  0.006 -0.016   
               1880   0.000 0.046 -0.024  0.011  0.108  0.026  0.005 -0.010   
               1890   0.000 0.056 -0.031  0.013  0.122  0.040  0.005 -0.013   
               1900   0.000 0.064 -0.034  0.014  0.123  0.055  0.009 -0.016   
               1910   0.000 0.051 -0.026  0.011  0.113  0.032  0.006 -0.011   
               1920   0.000 0.059 -0.031  0.012  0.129  0.036  0.005 -0.012   
               1930   0.001 0.065 -0.030  0.012  0.153  0.017  0.012 -0.008   
               1940   0.001 0.070 -0.032  0.009  0.141  0.028  0.009 -0.008   
               1950   0.000 0.084 -0.038  0.012  0.228  0.002  0.010  0.002   
               1960   0.001 0.163 -0.026  0.024  0.351 -0.003  0.081 -0.004   
               1970   0.001 0.112 -0.018  0.016  0.224  0.001  0.046  0.003   
               1980   0.001 0.097 -0.039  0.011  0.205  0.002  0.005  0.000   
               1990   0.001 0.104 -0.056  0.007  0.200  0.024  0.003  0.007   
aa_m           1800   0.000 0.008 -0.006  0.003  0.092 -0.048 -0.005  0.007   
               1810   0.000 0.012 -0.009  0.004  0.141 -0.074 -0.007  0.011   
               1820   0.000 0.009 -0.007  0.003  0.103 -0.054 -0.005  0.008   
               1830   0.000 0.009 -0.007  0.003  0.105 -0.055 -0.005  0.008   
               1850   0.000 0.010 -0.007  0.003  0.112 -0.058 -0.006  0.009   
               1860   0.000 0.011 -0.008  0.003  0.123 -0.064 -0.006  0.010   
               1870   0.000 0.012 -0.009  0.004  0.138 -0.072 -0.007  0.011   
               1880   0.000 0.008 -0.006  0.003  0.095 -0.050 -0.005  0.007   
               1900   0.000 0.009 -0.006  0.003  0.103 -0.054 -0.005  0.008   
               1910   0.000 0.011 -0.008  0.004  0.133 -0.070 -0.007  0.010   
...                     ...   ...    ...    ...    ...    ...    ...    ...   
achievements_m 1910   0.000 0.004 -0.003 -0.001  0.016 -0.008 -0.001  0.001   
               1920   0.000 0.004 -0.003 -0.001  0.016 -0.008 -0.001  0.001   
               1940   0.000 0.004 -0.003 -0.001  0.016 -0.008 -0.001  0.001   
               1950   0.000 0.019 -0.009  0.008  0.036  0.036  0.012  0.021   
               1960   0.000 0.023 -0.012  0.010  0.086  0.008  0.009  0.023   
               1970   0.000 0.021 -0.013  0.009  0.125 -0.031  0.002  0.019   
               1980   0.000 0.023 -0.013  0.010  0.105 -0.007  0.007  0.022   
               1990   0.000 0.016 -0.011  0.007  0.130 -0.051 -0.002  0.013   
achieving_m    1950   0.000 0.001 -0.001 -0.001  0.002  0.002  0.002  0.000   
               1960   0.000 0.003 -0.002 -0.000  0.007 -0.001  0.001  0.000   
               1970   0.000 0.003 -0.002 -0.001  0.003  0.000  0.001  0.003   
               1980   0.000 0.021 -0.015 -0.011  0.004 -0.006 -0.002  0.019   
               1990   0.000 0.023 -0.011 -0.006 -0.000  0.003 -0.004  0.012   
achilles_m     1800   0.000 0.007 -0.006 -0.005 -0.002 -0.001 -0.001  0.000   
               1830   0.000 0.388 -0.219  0.068  0.374  0.590  0.058 -0.142   
               1850   0.000 0.385 -0.217  0.070  0.375  0.592  0.058 -0.142   
               1860   0.000 0.378 -0.211  0.076  0.377  0.597  0.060 -0.143   
               1870   0.000 0.383 -0.215  0.072  0.375  0.594  0.059 -0.14

In [74]:
t1=time.time()


6.461143493652344e-05

    77.549


In [33]:
    compounds_reduced=df_reduced.loc[df_reduced['common'].str.endswith(r'_n')]
    compounds_reduced['modifier'],compounds_reduced['head']=compounds_reduced['common'].str.split(' ', 1).str
    compounds_reduced.drop(['common'],axis=1,inplace=True)
    compounds_reduced

KeyError: 'common'

In [15]:
heads=co_occ_reduced.loc[co_occ_reduced.constituent.str.endswith(r'_h')]
heads['constituent']=heads['constituent'].str.replace(r'_h$', r'_n', regex=True)
heads.to_csv('/data/dharp/compounding/datasets/heads_context_decadal_svd.csv',index=False,header=False,sep='\t')

In [16]:
modifiers=co_occ_reduced.loc[co_occ_reduced.constituent.str.endswith(r'_m')]
modifiers['constituent']=modifiers['constituent'].str.replace(r'_m$', r'_n', regex=True)
modifiers.to_csv('/data/dharp/compounding/datasets/modifiers_context_decadal_svd.csv',index=False,header=False,sep='\t')